# 第 6 章. Trino

## Trinoでデータ処理を実行する

Trino は複数のデータソースに対する統一的な SQL インターフェースを提供する分散 SQL クエリエンジンです。大規模なデータウェアハウスからデータレイク、さらには複数のリレーショナルデータベースまで、異なるデータソースを単一のクエリで結合できます。このノートブックでは、Trino と Iceberg を組み合わせた利用方法を実践的に学びます。

### 1. 初期設定

In [ ]:
# 必要なライブラリのインストール
!pip install -q trino pandas

In [ ]:
# 環境変数の設定
TRINO_URI = "http://trino:8085"
WAREHOUSE = "s3://amzn-s3-demo-bucket"

### 2. Trinoへの接続

Trino Python クライアントを使用して、Trino サーバーに接続します。

In [ ]:
from trino.dbapi import connect
import pandas as pd

# Icebergカタログを使用してTrinoに接続
conn = connect(host=TRINO_URI, user="trino", catalog="iceberg")
cur = conn.cursor()

# カタログの確認
cur.execute("SHOW CATALOGS")
catalogs = cur.fetchall()
print("利用可能なカタログ:")
for catalog in catalogs:
    print(catalog[0])

## 基本的なクエリ実行例

### 1. スキーマの作成

まずは新しいスキーマを作成します。

In [ ]:
# スキーマの作成
cur.execute("CREATE SCHEMA IF NOT EXISTS iceberg.retail")

# スキーマ一覧の表示
cur.execute("SHOW SCHEMAS FROM iceberg")
schemas = cur.fetchall()
print("Icebergカタログ内のスキーマ:")
for schema in schemas:
    print(schema[0])

### 2. テーブルの作成

商品情報を格納するテーブルを作成します。Icebergのテーブル形式はParquetをデフォルトとして使用します。

In [ ]:
# 商品テーブルの作成
cur.execute("""
CREATE OR REPLACE TABLE iceberg.retail.products (
    product_id VARCHAR,
    name VARCHAR,
    category VARCHAR,
    price DECIMAL(10, 2),
    created_at TIMESTAMP
)
""")

# テーブル構造の確認
cur.execute("DESCRIBE iceberg.retail.products")
table_schema = cur.fetchall()
pd.DataFrame(table_schema, columns=['Column', 'Type', 'Extra', 'Comment'])

### 3. データの挿入

作成したテーブルにサンプルデータを挿入します。

In [ ]:
# データの挿入
cur.execute("""
INSERT INTO iceberg.retail.products VALUES
    ('P001', 'Laptop', 'Electronics', 1299.99, TIMESTAMP '2023-01-15 10:00:00'),
    ('P002', 'Smartphone', 'Electronics', 899.99, TIMESTAMP '2023-01-16 11:30:00'),
    ('P003', 'Coffee Maker', 'Kitchen', 89.99, TIMESTAMP '2023-01-17 09:15:00'),
    ('P004', 'Blender', 'Kitchen', 49.99, TIMESTAMP '2023-01-18 14:20:00')
""")

# データの確認
cur.execute("SELECT * FROM iceberg.retail.products")
products = cur.fetchall()
columns = [desc[0] for desc in cur.description]
pd.DataFrame(products, columns=columns)

### 4. 集計クエリの実行

もう少し複雑なクエリを実行してみましょう。以下の例では、注文ステータス別に集計を行い、各ステータスの注文数と売上合計を取得します。

In [ ]:
# 注文ステータス別の集計
cur.execute("""
SELECT 
    status,
    COUNT(*) AS order_count,
    SUM(total_amount) AS total_sales
FROM iceberg.retail.orders
GROUP BY status
""")
status_summary = cur.fetchall()
status_columns = [desc[0] for desc in cur.description]
pd.DataFrame(status_summary, columns=status_columns)

## Iceberg の利用を開始する

### 1. テーブルの作成と変更

まずは基本的なテーブル作成と変更のオペレーションを見ていきましょう。

In [ ]:
cur.execute("""
CREATE OR REPLACE TABLE iceberg.retail.customers (
    customer_id BIGINT,
    name VARCHAR,
    email VARCHAR,
    registration_date DATE,
    segment VARCHAR,
    active BOOLEAN
)
""")

# テーブル情報の確認
cur.execute("DESCRIBE iceberg.retail.customers")
table_schema = cur.fetchall()
pd.DataFrame(table_schema, columns=['Column', 'Type', 'Extra', 'Comment'])

In [ ]:
# テーブル変更：新しい列の追加
cur.execute("ALTER TABLE iceberg.retail.customers ADD COLUMN birth_date DATE")
cur.execute("DESCRIBE iceberg.retail.customers")
updated_schema = cur.fetchall()
pd.DataFrame(updated_schema, columns=['Column', 'Type', 'Extra', 'Comment'])

In [ ]:
# テーブル変更：列の型変更
cur.execute("ALTER TABLE iceberg.retail.customers ALTER COLUMN name SET DATA TYPE VARCHAR(100)")

# テーブル変更：列名の変更
cur.execute("ALTER TABLE iceberg.retail.customers RENAME COLUMN email TO email_address")

# 最終的なテーブルスキーマの確認
cur.execute("DESCRIBE iceberg.retail.customers")
final_schema = cur.fetchall()
pd.DataFrame(final_schema, columns=['Column', 'Type', 'Extra', 'Comment'])

### 2. CREATE TABLE AS SELECTを使用したテーブル作成

Trino では、既存のクエリ結果から直接新しい Iceberg テーブルを作成できる「CREATE TABLE AS SELECT」（CTAS）構文をサポートしています。これは、ETL パイプラインや分析結果の保存に非常に便利です。

In [ ]:
# CTAS　の元になるテーブルを作成
cur.execute("""
CREATE OR REPLACE TABLE iceberg.retail.sales_by_date (
    sale_id VARCHAR,
    product_id VARCHAR,
    customer_id BIGINT,
    sale_date DATE,
    quantity INTEGER,
    amount DECIMAL(10, 2),
    region VARCHAR
)
WITH (
    format = 'PARQUET',
    partitioning = ARRAY['sale_date']
)
""")

# データの挿入
cur.execute("""
INSERT INTO iceberg.retail.sales_by_date VALUES
    ('S-1001', 'P-101', 1001, DATE '2023-01-15', 2, 129.99, 'North'),
    ('S-1002', 'P-102', 1002, DATE '2023-01-15', 1, 89.95, 'North'),
    ('S-1003', 'P-101', 1003, DATE '2023-01-16', 3, 194.98, 'South'),
    ('S-1004', 'P-103', 1001, DATE '2023-02-01', 1, 49.99, 'North'),
    ('S-1005', 'P-102', 1004, DATE '2023-02-01', 2, 179.90, 'East'),
    ('S-1006', 'P-101', 1002, DATE '2023-02-15', 1, 64.99, 'North'),
    ('S-1007', 'P-103', 1003, DATE '2023-02-20', 4, 199.96, 'South'),
    ('S-1008', 'P-104', 1005, DATE '2023-03-01', 2, 149.98, 'West'),
    ('S-1009', 'P-102', 1002, DATE '2023-03-05', 3, 269.85, 'North'),
    ('S-1010', 'P-101', 1001, DATE '2023-03-15', 1, 64.99, 'North')
""")

# データの確認
cur.execute("SELECT * FROM iceberg.retail.sales_by_date")
products = cur.fetchall()
columns = [desc[0] for desc in cur.description]
pd.DataFrame(products, columns=columns)

In [ ]:
# sales_by_dateテーブルから月次売上サマリーを作成
cur.execute("""
CREATE OR REPLACE TABLE iceberg.retail.monthly_sales_summary
WITH (
    format = 'PARQUET',
    partitioning = ARRAY['region']
)
AS
SELECT 
    date_trunc('month', sale_date) AS month,
    region,
    COUNT(*) AS transaction_count,
    SUM(amount) AS total_sales
FROM iceberg.retail.sales_by_date
GROUP BY date_trunc('month', sale_date), region
""")

# 作成されたテーブルのデータを確認
cur.execute("SELECT * FROM iceberg.retail.monthly_sales_summary ORDER BY month, region")
summary_data = cur.fetchall()
columns = [desc[0] for desc in cur.description]
pd.DataFrame(summary_data, columns=columns)

### 3. パーティションを利用したテーブルの作成

Icebergはパーティショニングをサポートしています。ここでは、注文データをステータスでパーティショニングしたテーブルを作成します。

In [ ]:
# 注文テーブルの作成（ステータスでパーティショニング）
cur.execute("""
CREATE OR REPLACE TABLE iceberg.retail.orders (
    order_id VARCHAR,
    customer_id BIGINT,
    order_date DATE,
    total_amount DECIMAL(10, 2),
    status VARCHAR
)
WITH (
    format = 'PARQUET',
    partitioning = ARRAY['status']
)
""")

# サンプルデータの挿入
cur.execute("""
INSERT INTO iceberg.retail.orders VALUES
    ('O-1001', 101, DATE '2023-02-01', 149.99, 'COMPLETED'),
    ('O-1002', 102, DATE '2023-02-01', 89.95, 'PROCESSING'),
    ('O-1003', 101, DATE '2023-02-02', 32.50, 'SHIPPED'),
    ('O-1004', 103, DATE '2023-02-03', 199.99, 'PROCESSING'),
    ('O-1005', 101, DATE '2023-02-05', 75.00, 'COMPLETED')
""")

# データの確認
cur.execute("SELECT * FROM iceberg.retail.orders")
orders = cur.fetchall()
order_columns = [desc[0] for desc in cur.description]
pd.DataFrame(orders, columns=order_columns)

Icebergのメタデータテーブルを使って、パーティション情報を確認できます。

In [ ]:
# パーティション情報の確認
cur.execute("SELECT * FROM iceberg.retail.\"orders$partitions\"")
partitions = cur.fetchall()
partition_columns = [desc[0] for desc in cur.description]
pd.DataFrame(partitions, columns=partition_columns)

パーティション列を使った集計クエリを実行します。パーティショニングにより、特定のステータスに関するクエリが効率的に実行されます。

In [ ]:
# 注文ステータス別の集計
cur.execute("""
SELECT 
    status,
    COUNT(*) AS order_count,
    SUM(total_amount) AS total_sales
FROM iceberg.retail.orders
GROUP BY status
""")
status_summary = cur.fetchall()
status_columns = [desc[0] for desc in cur.description]
pd.DataFrame(status_summary, columns=status_columns)

### 4. Transform を使用したパーティショニング
Iceberg はシンプルな列ベースのパーティショニングだけでなく、Transform を用いた高度なパーティショニング戦略もサポートしています。これらの関数を使うことで、データ分布に最適化されたパーティショニングスキーマを構築できます。
Trino でサポートされる主なパーティション変換関数は以下の通りです：

- 時間関連の変換関数: year(), month(), day(), hour()
- バケッティング関数: bucket(バケット数, 列名)
- 値の切り捨て関数: truncate(長さ, 列名)

以下の例では、これらの変換関数を活用した様々なパーティショニング戦略を実装します：

In [ ]:
# 1. 時間階層パーティショニング - 年と月でパーティション分割
cur.execute("""
CREATE OR REPLACE TABLE iceberg.retail.time_hierarchical_sales (
    sale_id VARCHAR,
    product_id VARCHAR,
    customer_id BIGINT,
    sale_timestamp TIMESTAMP,
    amount DECIMAL(10, 2)
)
WITH (
    format = 'PARQUET',
    partitioning = ARRAY['month(sale_timestamp)']
)
""")

# 2. 数値範囲のバケッティング - 金額を範囲でバケット化
cur.execute("""
CREATE OR REPLACE TABLE iceberg.retail.amount_bucketed_sales (
    sale_id VARCHAR,
    product_id VARCHAR,
    customer_id BIGINT,
    sale_date DATE,
    amount DECIMAL(10, 2)
)
WITH (
    format = 'PARQUET',
    partitioning = ARRAY['truncate(amount,1000)']  -- 1000単位で金額をグループ化
)
""")

# 3. 複合パーティショニング - 地域によるパーティショニングと顧客IDのハッシュバケッティング
cur.execute("""
CREATE OR REPLACE TABLE iceberg.retail.region_customer_sales (
    sale_id VARCHAR,
    product_id VARCHAR,
    customer_id BIGINT,
    sale_date DATE,
    region VARCHAR,
    amount DECIMAL(10, 2)
)
WITH (
    format = 'PARQUET',
    partitioning = ARRAY['region', 'bucket(customer_id, 10)']  -- 地域によるパーティショニングと顧客 ID のバケット分割を設定
)
""")

## 読み込みオペレーション

### 1. 基本的なクエリと条件付きフィルタリング

テーブルデータの基本的な読み取りとフィルタリングの例を見てみましょう：

In [ ]:
cur.execute("""
CREATE OR REPLACE TABLE iceberg.retail.sales_by_date (
    sale_id VARCHAR,
    product_id VARCHAR,
    customer_id BIGINT,
    sale_date DATE,
    quantity INTEGER,
    amount DECIMAL(10, 2),
    region VARCHAR
)
WITH (
    format = 'PARQUET',
    partitioning = ARRAY['sale_date']
)
""")

# データの挿入
cur.execute("""
INSERT INTO iceberg.retail.sales_by_date VALUES
    ('S-1001', 'P-101', 1001, DATE '2023-01-15', 2, 129.99, 'North'),
    ('S-1002', 'P-102', 1002, DATE '2023-01-15', 1, 89.95, 'North'),
    ('S-1003', 'P-101', 1003, DATE '2023-01-16', 3, 194.98, 'South'),
    ('S-1004', 'P-103', 1001, DATE '2023-02-01', 1, 49.99, 'North'),
    ('S-1005', 'P-102', 1004, DATE '2023-02-01', 2, 179.90, 'East'),
    ('S-1006', 'P-101', 1002, DATE '2023-02-15', 1, 64.99, 'North'),
    ('S-1007', 'P-103', 1003, DATE '2023-02-20', 4, 199.96, 'South'),
    ('S-1008', 'P-104', 1005, DATE '2023-03-01', 2, 149.98, 'West'),
    ('S-1009', 'P-102', 1002, DATE '2023-03-05', 3, 269.85, 'North'),
    ('S-1010', 'P-101', 1001, DATE '2023-03-15', 1, 64.99, 'North')
""")

# データの確認
cur.execute("SELECT * FROM iceberg.retail.sales_by_date")
products = cur.fetchall()
columns = [desc[0] for desc in cur.description]
pd.DataFrame(products, columns=columns)

In [ ]:
# 日付によるフィルタリング
# 2023年2月以降の販売データを取得
cur.execute("""
SELECT 
    sale_date,
    COUNT(*) AS sale_count,
    SUM(quantity) AS total_quantity,
    SUM(amount) AS total_amount
FROM iceberg.retail.sales_by_date
WHERE sale_date >= DATE '2023-02-01'
GROUP BY sale_date
ORDER BY sale_date
""")
filtered_data = cur.fetchall()
filtered_columns = [desc[0] for desc in cur.description]
pd.DataFrame(filtered_data, columns=filtered_columns)

In [ ]:
# 複合条件によるフィルタリング
# 2023年1月の100ドル以上の販売を製品別に集計
cur.execute("""
SELECT 
    product_id,
    SUM(quantity) AS total_quantity,
    SUM(amount) AS total_amount
FROM iceberg.retail.sales_by_date
WHERE sale_date BETWEEN DATE '2023-01-01' AND DATE '2023-01-31'
  AND amount > 100.00
GROUP BY product_id
""")
product_summary = cur.fetchall()
product_columns = [desc[0] for desc in cur.description]
pd.DataFrame(product_summary, columns=product_columns)

### 2. データの更新とタイムトラベルクエリの実行

Iceberg の重要な機能の一つであるタイムトラベルを使って、テーブルの過去のバージョンにアクセスする方法を見ていきましょう。

In [ ]:
# タイムトラベルのデモ用テーブル作成
cur.execute("""
CREATE OR REPLACE TABLE iceberg.retail.time_travel_demo (
    id INTEGER,
    name VARCHAR,
    value DOUBLE
)
""")

# 最初のデータセットを挿入
cur.execute("""
INSERT INTO iceberg.retail.time_travel_demo VALUES
    (1, 'Item A', 10.5),
    (2, 'Item B', 20.75),
    (3, 'Item C', 15.25)
""")

# 現在のタイムスタンプを記録
cur.execute("SELECT current_timestamp")
first_timestamp = cur.fetchone()[0]
print(f"最初のデータセット挿入時刻: {first_timestamp}")

In [ ]:
# テーブルスナップショットの確認
cur.execute("SELECT * FROM iceberg.retail.\"time_travel_demo$snapshots\"")
snapshots_after_first = cur.fetchall()
first_snapshot_id = snapshots_after_first[0][1]  # snapshot_id
print(f"最初のスナップショットID: {first_snapshot_id}")

In [ ]:
# 2つ目のデータセットを挿入
cur.execute("""
INSERT INTO iceberg.retail.time_travel_demo VALUES
    (4, 'Item D', 30.0),
    (5, 'Item E', 25.5)
""")

# 現在のテーブルデータを確認
cur.execute("SELECT * FROM iceberg.retail.time_travel_demo ORDER BY id")
current_data = cur.fetchall()
current_columns = [desc[0] for desc in cur.description]
pd.DataFrame(current_data, columns=current_columns)

In [ ]:
# タイムスタンプによるタイムトラベル
cur.execute(f"""
SELECT * FROM iceberg.retail.time_travel_demo FOR TIMESTAMP AS OF TIMESTAMP '{first_timestamp}'
ORDER BY id
""")
first_version_by_time = cur.fetchall()
pd.DataFrame(first_version_by_time, columns=current_columns)

In [ ]:
# スナップショットIDによるタイムトラベル
cur.execute(f"""
SELECT * FROM iceberg.retail.time_travel_demo FOR VERSION AS OF {first_snapshot_id}
ORDER BY id
""")
first_version_by_id = cur.fetchall()
pd.DataFrame(first_version_by_id, columns=current_columns)

In [ ]:
# スナップショット履歴の確認
cur.execute("SELECT * FROM iceberg.retail.\"time_travel_demo$history\"")
history = cur.fetchall()
history_columns = [desc[0] for desc in cur.description]
pd.DataFrame(history, columns=history_columns)

### 3. 高度なWindowクエリを使用した分析

Window関数はデータの集計と分析に非常に強力なツールです。行のグループ（ウィンドウ）に対する計算を行い、各行に対して結果を返します。詳細なデータを保持したまま集計値を計算できる点が大きな利点です。

In [ ]:
# 詳細な販売データテーブルを作成
cur.execute("""
CREATE OR REPLACE TABLE iceberg.retail.detailed_sales (
    sale_id VARCHAR,
    sale_date DATE,
    product_id VARCHAR,
    product_category VARCHAR,
    store_id VARCHAR,
    sales_person_id VARCHAR,
    quantity INTEGER,
    unit_price DECIMAL(10, 2),
    discount DECIMAL(5, 2),
    total_price DECIMAL(10, 2)
)
""")

# サンプルデータの挿入
cur.execute("""
INSERT INTO iceberg.retail.detailed_sales VALUES
    ('S-1001', DATE '2023-01-05', 'P-101', 'Electronics', 'Store-A', 'SP-1', 2, 500.00, 0.00, 1000.00),
    ('S-1002', DATE '2023-01-10', 'P-102', 'Home', 'Store-B', 'SP-2', 1, 300.00, 30.00, 270.00),
    ('S-1003', DATE '2023-01-15', 'P-101', 'Electronics', 'Store-A', 'SP-1', 1, 500.00, 50.00, 450.00),
    ('S-1004', DATE '2023-01-20', 'P-103', 'Clothing', 'Store-C', 'SP-3', 3, 100.00, 0.00, 300.00),
    ('S-1005', DATE '2023-01-25', 'P-104', 'Home', 'Store-B', 'SP-2', 2, 250.00, 25.00, 475.00),
    ('S-1006', DATE '2023-02-05', 'P-101', 'Electronics', 'Store-A', 'SP-1', 1, 520.00, 0.00, 520.00),
    ('S-1007', DATE '2023-02-10', 'P-105', 'Clothing', 'Store-C', 'SP-3', 4, 80.00, 0.00, 320.00),
    ('S-1008', DATE '2023-02-15', 'P-102', 'Home', 'Store-B', 'SP-4', 2, 300.00, 60.00, 540.00),
    ('S-1009', DATE '2023-02-20', 'P-103', 'Clothing', 'Store-A', 'SP-1', 1, 100.00, 10.00, 90.00),
    ('S-1010', DATE '2023-02-25', 'P-104', 'Home', 'Store-B', 'SP-2', 3, 250.00, 0.00, 750.00)
""")

# 1. カテゴリ別・月次のランキング集計
cur.execute("""
WITH monthly_product_sales AS (
    SELECT 
        DATE_TRUNC('month', sale_date) AS sale_month,
        product_id,
        product_category AS category,
        SUM(total_price) AS monthly_sales
    FROM iceberg.retail.detailed_sales
    GROUP BY DATE_TRUNC('month', sale_date), product_id, product_category
)
SELECT 
    sale_month,
    product_id,
    category,
    monthly_sales,
    RANK() OVER (
        PARTITION BY sale_month, category 
        ORDER BY monthly_sales DESC
    ) AS category_rank
FROM monthly_product_sales
ORDER BY sale_month, category, category_rank
""")

rank_data = cur.fetchall()
rank_columns = [desc[0] for desc in cur.description]
pd.DataFrame(rank_data, columns=rank_columns)
print("\n製品カテゴリ別・月次ランキング:")
print(rank_df)

# 2. 月次と累積売上
cur.execute("""
WITH monthly_category_sales AS (
    SELECT 
        DATE_TRUNC('month', sale_date) AS sale_month,
        product_category AS category,
        SUM(total_price) AS monthly_total
    FROM iceberg.retail.detailed_sales
    GROUP BY DATE_TRUNC('month', sale_date), product_category
)
SELECT 
    sale_month,
    category,
    monthly_total,
    SUM(monthly_total) OVER (
        PARTITION BY category 
        ORDER BY sale_month
        ROWS UNBOUNDED PRECEDING
    ) AS cumulative_sales
FROM monthly_category_sales
ORDER BY category, sale_month
""")

cum_data = cur.fetchall()
cum_columns = [desc[0] for desc in cur.description]
cum_df = pd.DataFrame(cum_data, columns=cum_columns)
print("\n月次売上合計と累積売上:")
print(cum_df)

# 3. 製品ごとの単価の移動平均と前後の値
cur.execute("""
WITH product_prices AS (
    SELECT DISTINCT
        product_id,
        DATE_TRUNC('month', sale_date) AS sale_month,
        AVG(unit_price) AS unit_price
    FROM iceberg.retail.detailed_sales
    GROUP BY product_id, DATE_TRUNC('month', sale_date)
)
SELECT 
    product_id,
    sale_month,
    unit_price,
    LAG(unit_price, 1) OVER (PARTITION BY product_id ORDER BY sale_month) AS prev_price,
    LEAD(unit_price, 1) OVER (PARTITION BY product_id ORDER BY sale_month) AS next_price,
    AVG(unit_price) OVER (
        PARTITION BY product_id 
        ORDER BY sale_month
        ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING
    ) AS rolling_avg_price
FROM product_prices
ORDER BY product_id, sale_month
""")

price_data = cur.fetchall()
price_columns = [desc[0] for desc in cur.description]
price_df = pd.DataFrame(price_data, columns=price_columns)
print("\n製品別の移動平均価格:")
print(price_df)

### 4. メタデータテーブルの探索

Iceberg はテーブルに関する多くのメタデータを提供しており、これを参照することでテーブルの状態や履歴を把握できます。以下の例では、Iceberg テーブルのメタデータテーブルをクエリして、テーブルの内部構造、履歴、ファイル情報などを表示しています。これらのメタデータは、テーブル管理やトラブルシューティングに役立ちます。メタデータテーブルにアクセスする際は、テーブル名に特殊なサフィックスを付けてクエリします。例えば、`table_name$properties` や `table_name$snapshots` などです。

In [ ]:
# メタデータテーブルの一覧を表示
metadata_tables = [
    "properties",
    "history",
    "metadata_log_entries",
    "snapshots",
    "manifests",
    "all_manifests", 
    "partitions",
    "files",
    "entries",
    "all_entries",
    "refs"
]

# 各メタデータテーブルを順に表示
for metadata in metadata_tables:
    print(f"\n{metadata} メタデータテーブル:")
    try:
        cur.execute(f"SELECT * FROM iceberg.retail.\"sales_by_date${metadata}\" LIMIT 5")
        data = cur.fetchall()
        if data:
            columns = [desc[0] for desc in cur.description]
            df = pd.DataFrame(data, columns=columns)
            print(df)
        else:
            print("データがありません")
    except Exception as e:
        print(f"エラー: {e}")

## 書き込みオペレーション

### 1. データの更新と削除

Iceberg テーブルのデータを更新・削除する例を見てみましょう。ここでは、`INSERT INTO` ステートメントを使用して新しいデータを追加し、`UPDATE` と `DELETE` ステートメントを使用して既存のデータを変更します。  

In [ ]:
cur.execute("""
CREATE OR REPLACE TABLE iceberg.retail.inventory (
    product_id VARCHAR,
    name VARCHAR,
    category VARCHAR,
    stock INTEGER,
    unit_price DECIMAL(10, 2),
    last_updated TIMESTAMP
)
""")

# サンプルデータの挿入
cur.execute("""
INSERT INTO iceberg.retail.inventory VALUES
    ('P001', 'Laptop', 'Electronics', 50, 999.99, TIMESTAMP '2023-01-01 00:00:00'),
    ('P002', 'Smartphone', 'Electronics', 100, 499.99, TIMESTAMP '2023-01-01 00:00:00'),
    ('P003', 'Headphones', 'Electronics', 200, 99.99, TIMESTAMP '2023-01-01 00:00:00'),
    ('P004', 'Coffee Maker', 'Kitchen', 30, 79.99, TIMESTAMP '2023-01-01 00:00:00'),
    ('P005', 'Blender', 'Kitchen', 45, 49.99, TIMESTAMP '2023-01-01 00:00:00')
""")

# 初期データの確認
cur.execute("SELECT * FROM iceberg.retail.inventory ORDER BY product_id")
initial_data = cur.fetchall()
columns = [desc[0] for desc in cur.description]
pd.DataFrame(initial_data, columns=columns)

In [ ]:
# データの更新: 価格の調整と在庫の更新
cur.execute("""
UPDATE iceberg.retail.inventory
SET 
    stock = stock - 10,
    unit_price = unit_price * 0.9,  -- 10%割引
    last_updated = current_timestamp
WHERE category = 'Electronics'
""")

In [ ]:
# 特定の製品の削除
cur.execute("""
DELETE FROM iceberg.retail.inventory
WHERE product_id = 'P005'
""")

In [ ]:
# 更新後のデータを確認
cur.execute("SELECT * FROM iceberg.retail.inventory ORDER BY product_id")
updated_data = cur.fetchall()
pd.DataFrame(updated_data, columns=columns)

In [ ]:
# テーブル履歴の確認
cur.execute("SELECT * FROM iceberg.retail.\"inventory$history\"")
history = cur.fetchall()
history_columns = [desc[0] for desc in cur.description]
history_df = pd.DataFrame(history, columns=history_columns)
print("\nテーブル変更履歴:")
print(history_df)

### 2. MERGEによるUpsert操作

MERGEステートメントは、条件に基づいてデータの更新と挿入を一度に行う強力な機能です。典型的なETLパイプラインでよく使用されます。

In [ ]:
# 顧客データテーブルの作成
cur.execute("""
CREATE OR REPLACE TABLE iceberg.retail.customers (
    customer_id BIGINT,
    name VARCHAR,
    email VARCHAR,
    status VARCHAR,
    last_update TIMESTAMP
)
""")

# 初期データの挿入
cur.execute("""
INSERT INTO iceberg.retail.customers VALUES
    (101, '田中 太郎', 'tanaka@example.com', 'ACTIVE', TIMESTAMP '2023-01-01 10:00:00'),
    (102, '佐藤 花子', 'sato@example.com', 'ACTIVE', TIMESTAMP '2023-01-01 10:00:00'),
    (103, '鈴木 一郎', 'suzuki@example.com', 'INACTIVE', TIMESTAMP '2023-01-01 10:00:00')
""")

# 更新用テーブルの作成
cur.execute("""
CREATE OR REPLACE TABLE iceberg.retail.customer_updates (
    customer_id BIGINT,
    name VARCHAR,
    email VARCHAR,
    status VARCHAR
)
""")

# 更新データの挿入（既存顧客の更新と新規顧客）
cur.execute("""
INSERT INTO iceberg.retail.customer_updates VALUES
    (102, '佐藤 花子', 'sato.new@example.com', 'ACTIVE'),  -- メールアドレス更新
    (103, '鈴木 一郎', 'suzuki@example.com', 'ACTIVE'),    -- ステータス更新
    (104, '伊藤 誠', 'ito@example.com', 'ACTIVE')          -- 新規顧客
""")

# 元のデータを確認
cur.execute("SELECT customer_id, name, email, status FROM iceberg.retail.customers ORDER BY customer_id")
original_data = cur.fetchall()
original_columns = [desc[0] for desc in cur.description]
original_df = pd.DataFrame(original_data, columns=original_columns)
print("元の顧客データ:")
print(original_df)

# 更新データを確認
cur.execute("SELECT * FROM iceberg.retail.customer_updates ORDER BY customer_id")
updates = cur.fetchall()
update_columns = [desc[0] for desc in cur.description]
updates_df = pd.DataFrame(updates, columns=update_columns)
print("\n更新用データセット:")
print(updates_df)

# MERGE操作を実行
cur.execute("""
MERGE INTO iceberg.retail.customers c
USING iceberg.retail.customer_updates u
ON c.customer_id = u.customer_id
WHEN MATCHED THEN
    UPDATE SET 
        email = u.email,
        status = u.status,
        last_update = TIMESTAMP '2023-03-30 10:30:45.123'
WHEN NOT MATCHED THEN
    INSERT (customer_id, name, email, status, last_update)
    VALUES (u.customer_id, u.name, u.email, u.status, TIMESTAMP '2023-03-30 10:30:45.123')
""")

# MERGE後のデータを確認
cur.execute("SELECT * FROM iceberg.retail.customers ORDER BY customer_id")
merged_data = cur.fetchall()
merged_columns = [desc[0] for desc in cur.description]
merged_df = pd.DataFrame(merged_data, columns=merged_columns)
print("\nMERGE実行後の顧客データ:")
print(merged_df)

## プロシージャの利用

### 1. ファイルサイズの最適化

小さなファイルを統合して読み取りパフォーマンスを向上させることができます。

In [ ]:
cur.execute("""
CREATE OR REPLACE TABLE iceberg.retail.products (
    product_id VARCHAR,
    name VARCHAR,
    category VARCHAR,
    price DECIMAL(10, 2),
    created_at TIMESTAMP
)
""")

# 1つずつINSERTを実行して、意図的に小さなファイルを複数作成
products = [
    ('P001', 'Laptop', 'Electronics', 1299.99, "TIMESTAMP '2023-01-15 10:00:00'"),
    ('P002', 'Smartphone', 'Electronics', 899.99, "TIMESTAMP '2023-01-16 11:30:00'"),
    ('P003', 'Coffee Maker', 'Kitchen', 89.99, "TIMESTAMP '2023-01-17 09:15:00'"),
    ('P004', 'Blender', 'Kitchen', 49.99, "TIMESTAMP '2023-01-18 14:20:00'")
]

for product in products:
    cur.execute(f"""
    INSERT INTO iceberg.retail.products VALUES
        ('{product[0]}', '{product[1]}', '{product[2]}', {product[3]}, {product[4]})
    """)

# ファイル情報の確認
cur.execute("SELECT * FROM iceberg.retail.\"products$files\"")
files_before = cur.fetchall()
file_columns = [desc[0] for desc in cur.description]
files_df = pd.DataFrame(files_before, columns=file_columns)
print(f"最適化前のファイル数: {len(files_df)}")

# ファイル最適化の実行
try:
    cur.execute("ALTER TABLE iceberg.retail.products EXECUTE optimize")
    print("\nファイル最適化を実行しました。")
    
    # 最適化後のファイル情報を確認
    cur.execute("SELECT * FROM iceberg.retail.\"products$files\"")
    files_after = cur.fetchall()
    files_after_df = pd.DataFrame(files_after, columns=file_columns)
    print(f"\n最適化後のファイル数: {len(files_after_df)}")
except Exception as e:
    print(f"最適化中にエラーが発生しました: {e}")

### 2. テーブルのロールバックと履歴管理

Icebergの大きな特徴の一つは、テーブルの変更履歴を完全に保持し、特定の時点や特定のスナップショットにロールバックできることです。この機能はデータの誤った変更からの復旧や、監査目的で非常に役立ちます。

In [ ]:
# ロールバックデモ用のテーブルを作成
cur.execute("""
CREATE OR REPLACE TABLE iceberg.retail.rollback_demo (
    id INTEGER,
    name VARCHAR,
    value DOUBLE
)
""")

# 初期データを挿入
cur.execute("""
INSERT INTO iceberg.retail.rollback_demo VALUES
    (1, '商品A', 10.5),
    (2, '商品B', 20.8),
    (3, '商品C', 15.0)
""")

# 初期データを確認
cur.execute("SELECT * FROM iceberg.retail.rollback_demo ORDER BY id")
initial_data = cur.fetchall()
columns = [desc[0] for desc in cur.description]
initial_df = pd.DataFrame(initial_data, columns=columns)
print("初期状態のデータ:")
print(initial_df)

# スナップショット情報を確認
cur.execute("SELECT * FROM iceberg.retail.\"rollback_demo$snapshots\"")
snapshots = cur.fetchall()
snapshot_columns = [desc[0] for desc in cur.description]
snapshots_df = pd.DataFrame(snapshots, columns=snapshot_columns)
print("\n現在のスナップショット情報:")
print(snapshots_df)

# 最初のスナップショットIDを取得
first_snapshot_id = snapshots_df.iloc[0]['snapshot_id']
print(f"\n最初のスナップショットID: {first_snapshot_id}")

# データを更新
print("\nデータを更新中...")
# 既存データを更新
cur.execute("""
UPDATE iceberg.retail.rollback_demo
SET name = '商品A-更新', value = 15.5
WHERE id = 1
""")

cur.execute("""
UPDATE iceberg.retail.rollback_demo
SET name = '商品C-更新', value = 25.0
WHERE id = 3
""")

# 新しいデータを挿入
cur.execute("""
INSERT INTO iceberg.retail.rollback_demo VALUES
    (4, '商品D', 30.0)
""")

# 更新後のデータを確認
cur.execute("SELECT * FROM iceberg.retail.rollback_demo ORDER BY id")
updated_data = cur.fetchall()
updated_df = pd.DataFrame(updated_data, columns=columns)
print("更新後のデータ:")
print(updated_df)

# 更新後のスナップショット情報を確認
cur.execute("SELECT * FROM iceberg.retail.\"rollback_demo$snapshots\"")
updated_snapshots = cur.fetchall()
updated_snapshots_df = pd.DataFrame(updated_snapshots, columns=snapshot_columns)
print("\n更新後のスナップショット情報:")
print(updated_snapshots_df)

# 最初のスナップショットにロールバック
print(f"\n最初のスナップショットにロールバックします...")
cur.execute(f"""
ALTER TABLE iceberg.retail.rollback_demo
EXECUTE rollback_to_snapshot(snapshot_id => {first_snapshot_id})
""")

# ロールバック後のデータを確認
cur.execute("SELECT * FROM iceberg.retail.rollback_demo ORDER BY id")
rollback_data = cur.fetchall()
rollback_df = pd.DataFrame(rollback_data, columns=columns)
print("ロールバック後のデータ:")
print(rollback_df)

# テーブル操作の履歴を確認
cur.execute("SELECT * FROM iceberg.retail.\"rollback_demo$history\"")
history = cur.fetchall()
history_columns = [desc[0] for desc in cur.description]
history_df = pd.DataFrame(history, columns=history_columns)
print("\nテーブル操作履歴:")
print(history_df)

## まとめ

このノートブックでは、以下の内容を実践的に学びました：

1. **Trinoの基本**: Trinoクライアントの接続方法とクエリの実行
2. **Icebergテーブルの操作**: テーブルの作成、データの挿入と更新
3. **パーティショニング**: データをパーティションで効率的に管理する方法
4. **タイムトラベル**: 過去のデータスナップショットへのアクセス
5. **テーブルメンテナンス**: ファイルの最適化などのメンテナンスオペレーション

Trinoを使うことで、Icebergテーブルに対するSQLインターフェースが提供され、複雑なデータレイク環境でも馴染みのあるSQLを使って操作を実行できます。また、Icebergのバージョン管理、スキーマ進化、ACIDトランザクションなどの機能を同時に活用できます。